# macos+vscode+jupyter+clingという組み合わせでc++を扱う方法

1: cling + jupyterkernel（ここで紹介）
2: xeus-cling
3: root cling

最初２の方法を試したが，condaでclingがちゃんと入らない（入っても起動後にerrorになっちゃう）ので，1の方法を取ることにした．clingはbrewでもインストールできて，こちらのclingがちゃんと動いた．

clingが動いたら，jupyter用のkernelを別途インストールする必要がある．

```cpp
In file included from input_line_15:1:
In file included from /Library/Developer/CommandLineTools/SDKs/MacOSX12.sdk/usr/include/c++/v1/iostream:37:
In file included from /Library/Developer/CommandLineTools/SDKs/MacOSX12.sdk/usr/include/c++/v1/ios:214:
In file included from /Library/Developer/CommandLineTools/SDKs/MacOSX12.sdk/usr/include/c++/v1/__locale:15:
In file included from /Library/Developer/CommandLineTools/SDKs/MacOSX12.sdk/usr/include/c++/v1/string:519:
In file included from /Library/Developer/CommandLineTools/SDKs/MacOSX12.sdk/usr/include/c++/v1/__debug:14:
In file included from /Library/Developer/CommandLineTools/SDKs/MacOSX12.sdk/usr/include/c++/v1/iosfwd:98:
In file included from /Library/Developer/CommandLineTools/SDKs/MacOSX12.sdk/usr/include/c++/v1/__mbstate_t.h:29:
In file included from /Library/Developer/CommandLineTools/SDKs/MacOSX12.sdk/usr/include/c++/v1/wchar.h:123:
/Library/Developer/CommandLineTools/SDKs/MacOSX.sdk/usr/include/wchar.h:89:10: fatal error: 'stdarg.h' file not found
#include <stdarg.h>
         ^~~~~~~~~~
ename: evalue
```

となってiostreamがロードできない．これは困るので対策をする．jupyter notebookの場合，単にstdarg.hがあるディレクトリにcpathを通せばよかった．

```bash
export CPATH=/Library/Developer/CommandLineTools/usr/lib/tapi/14.0.3/include:$CPATH
```

しかし，この方法だとvscodeでは解決せず，結局`/Library/Developer/CommandLineTools/SDKs/MacOSX.sdk/usr/include/`ディレクトリにstdarg.hを手でリンクを貼ることで解決できた．


In [1]:
#include <stdio.h>
#include <iostream>
#include <vector>
using namespace std;

In [11]:
class AtomicCord // (x,y,z)の三次元ベクトル
{
  public:
    double position[3];
    //explicit AtomicCord(double x, double y, double z);  // コンストラクタ
    AtomicCord(double x, double y, double z) // in angstrom
  {
    position[0] = x;
    position[1] = y;
    position[2] = z;
  }; 
};

// AtomicCord::AtomicCord(double x, double y, double z)
// {
//   this->position = {x,y,z};
// };


In [19]:
AtomicCord classtest2 = AtomicCord(0.1,0.1,0.1);
std::cout << classtest2.position[0] << std::endl;

0.1


(std::basic_ostream &) @0x7ff85f2c7820


In [37]:
/*
 2023/5/30
 ase atomsに対応するAtomsクラスを定義する

どうも自作クラスをvectorに入れる場合は特殊な操作が必要な模様．
https://nprogram.hatenablog.com/entry/2017/07/05/073922
*/


class Atoms {
public: // public変数
  std::vector<int> atomic_num;
  std::vector<std::vector<double> > positions;
  std::vector<std::vector<double> > cell;
  std::vector<bool> pbc;
  // std::vector<int> get_atomic_numbers(); // atomic_numを返す
  
  int number;
  Atoms(std::vector<int> atomic_numbers,
        std::vector<std::vector<double> > atomic_positions,
        std::vector<std::vector<double> > UNITCELL_VECTORS,
        std::vector<bool> pbc = {true, true, true}
        )
        {
          for (int i = 0; i < atomic_numbers.size(); i++)
            { 
              atomic_num[i] = atomic_numbers[i];         
            }
          for (int i = 0; i < atomic_positions.size(); i++)
            { 
              positions[i] = atomic_positions[i];         
            }
          for (int i = 0; i < UNITCELL_VECTORS.size(); i++)
            { 
              cell[i] = UNITCELL_VECTORS[i];         
            }
          for (int i = 0; i < pbc.size(); i++)
            { 
              pbc[i] = pbc[i];         
            }
        };

  // std::vector<int> get_atomic_numbers()// atomic_numを返す
  // {
  //  return this->atomic_num;
  // };
};

// Atoms::Atoms(std::vector<int> atomic_num,
//         std::vector<vector<double>> positions,
//         std::vector<vector<double>> UNITCELL_VECTORS,
//         std::vector<bool> pbc = {true, true, true}
//         )
//         {
//           this->atomic_num = atomic_num;
//           this->positions = positions;
//           this->cell = cell;
//         };

// std::vector<int> Atoms::get_atomic_numbers()
// {
//   return this->atomic_num;
// };

/Library/Developer/CommandLineTools/SDKs/MacOSX12.sdk/usr/include/c++/v1/vector:365:5: error: constructor for 'std::__1::__vector_base<int, std::__1::allocator<int> >' must explicitly initialize the base class '__vector_base_common<true>' which does not have a default constructor
    __vector_base()
    ^
/Library/Developer/CommandLineTools/SDKs/MacOSX12.sdk/usr/include/c++/v1/vector:519:5: note: in instantiation of member function 'std::__1::__vector_base<int, std::__1::allocator<int> >::__vector_base' requested here
    vector() _NOEXCEPT_(is_nothrow_default_constructible<allocator_type>::value)
    ^
input_line_40:19:3: note: in instantiation of member function 'std::__1::vector<int, std::__1::allocator<int> >::vector' requested here
  Atoms(std::vector<int> atomic_numbers,
  ^
/Library/Developer/CommandLineTools/SDKs/MacOSX12.sdk/usr/include/c++/v1/vector:327:64: note: 'std::__1::__vector_base_common<true>' declared here
_LIBCPP_EXTERN_TEMPLATE(class _LIBCPP_EXTERN_TEMPLATE_TYPE_VI

ename: evalue

In [36]:
class Vector {
	vector<int> myVec;

public:
	Vector(vector<int> newVector) {
	    myVec = newVector;
	}
	
	void print() {
		for (int i = 0; i < myVec.size(); i++)
			cout << myVec[i] << " ";
	}
	
};


/Library/Developer/CommandLineTools/SDKs/MacOSX12.sdk/usr/include/c++/v1/vector:365:5: error: constructor for 'std::__1::__vector_base<int, std::__1::allocator<int> >' must explicitly initialize the base class '__vector_base_common<true>' which does not have a default constructor
    __vector_base()
    ^
/Library/Developer/CommandLineTools/SDKs/MacOSX12.sdk/usr/include/c++/v1/vector:519:5: note: in instantiation of member function 'std::__1::__vector_base<int, std::__1::allocator<int> >::__vector_base' requested here
    vector() _NOEXCEPT_(is_nothrow_default_constructible<allocator_type>::value)
    ^
input_line_39:5:2: note: in instantiation of member function 'std::__1::vector<int, std::__1::allocator<int> >::vector' requested here
        Vector(vector<int> newVector) {
        ^
/Library/Developer/CommandLineTools/SDKs/MacOSX12.sdk/usr/include/c++/v1/vector:327:64: note: 'std::__1::__vector_base_common<true>' declared here
_LIBCPP_EXTERN_TEMPLATE(class _LIBCPP_EXTERN_TEMPLATE_TYP

ename: evalue

In [ ]:
// export CPATH=/Library/Developer/CommandLineTools/usr/lib/tapi/14.0.3/include:$CPATH

In [8]:
int j;
for(int i = 1; i <= 5; i++) {
    j = i+1;
};

In [11]:
// export CPATH=/Library/Developer/CommandLineTools/usr/lib/tapi/14.0.3/include:$CPATH

input_line_14:3:5: error: redefinition of 'p'
int p;
    ^
input_line_13:3:5: note: previous definition is here
int p;
    ^


ename: evalue

In [2]:
using namespace std;

In [3]:
printf("Hello World!\n")

Hello World!


In file included from input_line_6:1:
In file included from /usr/local/Cellar/cling/0.9/libexec/include/cling/Interpreter/RuntimePrintValue.h:17:
In file included from /Library/Developer/CommandLineTools/SDKs/MacOSX12.sdk/usr/include/c++/v1/memory:671:
In file included from /Library/Developer/CommandLineTools/SDKs/MacOSX12.sdk/usr/include/c++/v1/__functional_base:26:
In file included from /Library/Developer/CommandLineTools/SDKs/MacOSX12.sdk/usr/include/c++/v1/utility:211:
In file included from /Library/Developer/CommandLineTools/SDKs/MacOSX12.sdk/usr/include/c++/v1/__debug:14:
In file included from /Library/Developer/CommandLineTools/SDKs/MacOSX12.sdk/usr/include/c++/v1/iosfwd:98:
In file included from /Library/Developer/CommandLineTools/SDKs/MacOSX12.sdk/usr/include/c++/v1/__mbstate_t.h:29:
In file included from /Library/Developer/CommandLineTools/SDKs/MacOSX12.sdk/usr/include/c++/v1/wchar.h:123:
/Library/Developer/CommandLineTools/SDKs/MacOSX.sdk/usr/include/wchar.h:89:10: fatal err

(int) 13


In [4]:
double calc(int s1, int s2) {
// ここはコメント行。s1 は消費支出，s2 は飲食費，エンゲル係数を返す。
    return s2 * 100. / s1;
}

In [5]:
calc(120000, 43000);

(double) 35.833333
